In [1]:
import os

In [2]:
%pwd

'd:\\github\\Github\\Wine-Quality-End-to-End-ML-Project\\notebook'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\github\\Github\\Wine-Quality-End-to-End-ML-Project'

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PowerTransformer, MinMaxScaler
from imblearn.over_sampling import SMOTE
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    scaler_name: str
    max_depth: int
    n_estimators: int
    random_state: int
    target_column: str

In [7]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.RandomForestClassifier
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            scaler_name = config.scaler_name, 
            max_depth = params.max_depth,
            n_estimators = params.n_estimators,
            random_state = params.random_state,            
            target_column = schema.name
            
        )

        return model_trainer_config

In [9]:
import pandas as pd
import os
from mlProject import logger
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import joblib

In [ ]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        # Load train and test data
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        # Split features and target
        X_train = train_data.drop([self.config.target_column], axis=1)
        X_test = test_data.drop([self.config.target_column], axis=1)
        y_train = train_data[self.config.target_column]
        y_test = test_data[self.config.target_column]

        # Define skewed and non-skewed features
        skewed_features = ['volatile acidity', 'citric acid', 'residual sugar', 'chlorides',  
                           'free sulfur dioxide', 'total sulfur dioxide', 'sulphates', 'alcohol']
        all_features = X_train.columns.tolist()
        other_features = [col for col in all_features if col not in skewed_features]

        # Wrap ColumnTransformer in a pipeline with MinMaxScaler
        preprocessing_pipeline = Pipeline([
            ('minmax_scaler', MinMaxScaler())
        ])

        # Apply pipeline to training and test sets
        X_train_processed = preprocessing_pipeline.fit_transform(X_train)
        X_test_processed = preprocessing_pipeline.transform(X_test)

        # Save the preprocessing pipeline
        joblib.dump(preprocessing_pipeline, os.path.join(self.config.root_dir, self.config.scaler_name))

        # Apply SMOTE to balance the training data
        smote = SMOTE(random_state=42, k_neighbors=2)
        X_train_resampled, y_train_resampled = smote.fit_resample(X_train_processed, y_train)

        # Train the RandomForestClassifier
        model = RandomForestClassifier(
            max_depth=self.config.max_depth,
            n_estimators=self.config.n_estimators,
            random_state=self.config.random_state
        )
        model.fit(X_train_resampled, y_train_resampled)

        # Save the trained model
        joblib.dump(model, os.path.join(self.config.root_dir, self.config.model_name))

In [11]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2025-04-15 11:33:20,265: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-04-15 11:33:20,268: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-15 11:33:20,272: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-04-15 11:33:20,275: INFO: common: created directory at: artifacts]
[2025-04-15 11:33:20,276: INFO: common: created directory at: artifacts/model_trainer]
